In [2]:
import tensorflow as tf
import os
from dlomix.data import load_processed_dataset
import numpy as np
from matplotlib import pyplot as plt
from dlomix.losses import masked_spectral_distance

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = ''

In [24]:
MODEL_DIR = '/cmnfs/proj/bmpc_dlomix/models/baseline_models/noptm_baseline_full_bs1024_unmod_extended/'
RUN_NAME = '7ef3360f-2349-46c0-a905-01187d4899e2'
model = tf.keras.models.load_model(MODEL_DIR + RUN_NAME + '.keras',)
print(model.summary())

(None, 29, 512)
Model: "prosit_intensity_predictor_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  464       
                                                                 
 sequential_5 (Sequential)   (None, 30, 512)           1996800   
                                                                 
 sequential_6 (Sequential)   multiple                  4608      
                                                                 
 sequential_7 (Sequential)   (None, 29, 512)           1576806   
                                                                 
 encoder_att (AttentionLaye  multiple                  542       
 r)                                                              
                                                                 
 sequential_8 (Sequential)   multiple                  0         
                      

In [6]:
ds = load_processed_dataset('/cmnfs/proj/bmpc_dlomix/datasets/processed/noptm_baseline_small_duplicated_ions')

In [7]:
batch, labels = next(iter(ds.tensor_train_data))

In [25]:
old_weights = model.layers[-1].get_layer('time_dense').get_weights()[0]

In [26]:
model.regressor = tf.keras.Sequential(
    [
        tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(3 * 4), name='time_dense'
            ), 
        tf.keras.layers.LeakyReLU(name='activation'), 
        tf.keras.layers.Flatten(name='out')], 
    name='regressor'
    )

In [27]:
model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001), loss=masked_spectral_distance)

In [28]:
model.fit(ds.tensor_test_data, validation_data=ds.tensor_test_data, epochs=1)

(None, 29, 512)
(None, 29, 512)
2/2 [==============================] - 13s 5s/step - loss: 0.8503 - val_loss: 0.7829


In [29]:
new_weights = model.get_layer('regressor').get_layer('time_dense').get_weights()[0]

In [31]:
np.allclose(old_weights, new_weights[:, :6])

False

In [32]:
import keras.backend as K

In [59]:
from tensorflow.keras.constraints import Constraint
@tf.keras.saving.register_keras_serializable()
class FixRegressorWeights(Constraint):
    def __init__(self, old_weights, old_fions):
        self.old_weights = old_weights
        self.freeze_weights = True
        self.old_fions = old_fions
    def __call__(self, w):
        if self.freeze_weights:
            return K.concatenate([self.old_weights, w[:, self.old_fions:]], axis=1)
        return w
    
class FixBias(Constraint):
    def __init__(self, old_bias, old_fions):
        self.old_bias = old_bias
        self.freeze_bias = True
        self.old_fions = old_fions
    def __call__(self, b):
        if self.freeze_bias:
            return K.concatenate([self.old_bias, b[self.old_fions:]], axis=0)
        return b

In [60]:
model = tf.keras.models.load_model(MODEL_DIR + RUN_NAME + '.keras')

(None, 29, 512)


In [61]:
old_weights = model.regressor.get_layer('time_dense').get_weights()[0]
old_bias = model.regressor.get_layer('time_dense').get_weights()[1]

In [62]:
fix_weights = FixRegressorWeights(old_weights, model.len_fion)
fix_bias = FixBias(old_bias, model.len_fion)

In [63]:

fix_weights = FixRegressorWeights(old_weights, model.len_fion)
fix_bias = FixBias(old_bias, model.len_fion)

model.regressor = tf.keras.Sequential(
    [
        tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(
                3 * 4, 
                kernel_constraint=fix_weights, 
                bias_constraint=fix_bias
                ), name='time_dense',
            ), 
        tf.keras.layers.LeakyReLU(name='activation'), 
        tf.keras.layers.Flatten(name='out')], 
    name='regressor'
    )

In [64]:
model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001), loss=masked_spectral_distance)

In [65]:
model.fit(ds.tensor_test_data, validation_data=ds.tensor_test_data, epochs=1)

(None, 29, 512)
(None, 29, 512)
2/2 [==============================] - 13s 5s/step - loss: 0.7581 - val_loss: 0.6037


In [66]:
new_weights = model.regressor.get_layer('time_dense').get_weights()[0]
new_bias = model.regressor.get_layer('time_dense').get_weights()[1]

In [67]:
np.allclose(old_weights, new_weights[:, :6])

True

In [68]:
np.allclose(old_bias, new_bias[:6])

True

In [73]:
model.regressor.get_layer('time_dense').layer.kernel_constraint.freeze_weights

True

In [74]:
model.regressor.get_layer('time_dense').layer.kernel_constraint.freeze_weights = False
model.regressor.get_layer('time_dense').layer.bias_constraint.freeze_bias = False

In [75]:
model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001), loss=masked_spectral_distance)

In [76]:
model.fit(ds.tensor_test_data, validation_data=ds.tensor_test_data, epochs=1)

(None, 29, 512)
(None, 29, 512)
2/2 [==============================] - 13s 5s/step - loss: 0.5999 - val_loss: 0.5829


In [78]:
old_weights, new_weights = new_weights, model.regressor.get_layer('time_dense').get_weights()[0]

In [80]:
np.allclose(old_weights[:, :6], new_weights[:, :6])

False

In [81]:
old_bias, new_bias = new_bias, model.regressor.get_layer('time_dense').get_weights()[1]

In [82]:
np.allclose(old_bias[:6], new_bias[:6])

False